In [2]:
import optuna
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("heart.csv")
df = pd.get_dummies(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Age                918 non-null    int64  
 1   RestingBP          918 non-null    int64  
 2   Cholesterol        918 non-null    int64  
 3   FastingBS          918 non-null    int64  
 4   MaxHR              918 non-null    int64  
 5   Oldpeak            918 non-null    float64
 6   HeartDisease       918 non-null    int64  
 7   Sex_F              918 non-null    bool   
 8   Sex_M              918 non-null    bool   
 9   ChestPainType_ASY  918 non-null    bool   
 10  ChestPainType_ATA  918 non-null    bool   
 11  ChestPainType_NAP  918 non-null    bool   
 12  ChestPainType_TA   918 non-null    bool   
 13  RestingECG_LVH     918 non-null    bool   
 14  RestingECG_Normal  918 non-null    bool   
 15  RestingECG_ST      918 non-null    bool   
 16  ExerciseAngina_N   918 non

In [4]:
X = df.drop("HeartDisease", axis=1).copy()
y = df["HeartDisease"].copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=369)

In [5]:
model = MLPClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)

0.8876811594202898


In [7]:
def objective(trial):
    hyperparams = {
        "hidden_layer_sizes": trial.suggest_categorical("hidden_layer_sizes", [
            (50,), (100,), (50, 50), (100, 50), (100, 100), (150, 100)
        ]),
        "activation": trial.suggest_categorical("activation", ["identity", "logistic", "tanh", "relu"]),
        "solver": trial.suggest_categorical("solver", ["lbfgs", "sgd", "adam"]),
        "alpha": trial.suggest_float("alpha", 1e-5, 1e-1, log=True),
        "batch_size": trial.suggest_categorical("batch_size", [32, 64, 128, 256]),
        "learning_rate": trial.suggest_categorical("learning_rate", ["constant", "invscaling", "adaptive"]),
        "learning_rate_init": trial.suggest_float("learning_rate_init", 1e-5, 1e-1, log=True),
        "power_t": trial.suggest_float("power_t", 0.1, 0.9), 
        "beta_1": trial.suggest_float("beta_1", 0.8, 0.999),  
        "beta_2": trial.suggest_float("beta_2", 0.8, 0.999), 
        "max_iter": 1000,  
        "early_stopping": trial.suggest_categorical("early_stopping", [True, False]),
        "n_iter_no_change": trial.suggest_int("n_iter_no_change", 5, 50),
    }

    if hyperparams["solver"] == "sgd":
        hyperparams["momentum"] = trial.suggest_float("momentum", 0.0, 0.99)
        hyperparams["nesterovs_momentum"] = trial.suggest_categorical("nesterovs_momentum", [True, False])
    model = MLPClassifier(**hyperparams, random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    return acc  

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, n_jobs=-1)

[I 2025-08-21 01:31:21,270] A new study created in memory with name: no-name-3f5b85b3-e31e-4284-bb3e-381f21c9cdac
[I 2025-08-21 01:31:22,177] Trial 8 finished with value: 0.4166666666666667 and parameters: {'hidden_layer_sizes': (50, 50), 'activation': 'identity', 'solver': 'adam', 'alpha': 0.017997560903271117, 'batch_size': 256, 'learning_rate': 'invscaling', 'learning_rate_init': 9.488849560089281e-05, 'power_t': 0.3321491488964773, 'beta_1': 0.9211346287098486, 'beta_2': 0.8392420769420478, 'early_stopping': False, 'n_iter_no_change': 6}. Best is trial 8 with value: 0.4166666666666667.
[I 2025-08-21 01:31:25,413] Trial 4 finished with value: 0.855072463768116 and parameters: {'hidden_layer_sizes': (50, 50), 'activation': 'tanh', 'solver': 'adam', 'alpha': 0.057718395900824816, 'batch_size': 256, 'learning_rate': 'constant', 'learning_rate_init': 0.014817657355694845, 'power_t': 0.4545249682107553, 'beta_1': 0.9559155780859716, 'beta_2': 0.9164253438694147, 'early_stopping': False, 

In [8]:
best_hiper = study.best_params
model = MLPClassifier(**best_hiper, random_state=369)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(acc)


0.9057971014492754
